In [28]:
import os
import sys

import branca.colormap as cm
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point, Polygon

sys.path.insert(0, "../src")
import json
import time as ti
import math

import geopandas as gpd
import seaborn as sns
from tqdm import tqdm
import seaborn as sns 
from IPython.display import display, Math, Latex
import requests

import urllib.request

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

# Now let's try to compute the distance OD Matrix between these points

First, let's define the function need to perform the HTTP GET request. All the assumptions regarding this matrix are under this first link below.

- Resources: https://docs.microsoft.com/en-us/bingmaps/rest-services/examples/distance-matrix-asynchronous-example

- Example: https://docs.microsoft.com/en-us/bingmaps/rest-services/examples/distance-matrix-example

The HTTP GET request works as follow:

https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins={lat0,long0;lat1,lon1;latM,lonM}&destinations={lat0,lon0;lat1,lon1;latN,longN}&travelMode={travelMode}&startTime={startTime}&timeUnit={timeUnit}&key={BingMapsKey}

- origins: list of lat, long coordinates of origins
- destionations: list of lat, long coordinates of destinations
- travelMode: `Driving`
- startTime: `2021-11-15-07:00`
- timeUnit
- BingMapsKey

In [29]:
def get_data(orig, dest, size_x, size_y):
    # Your Bing Maps Key
    bingMapsKey = "Ah52Budr_8V_sE1HKdPH-INIK_RDqas1zgkSJwdtB3GW3Dm04DsNm3lzkoXmpPc9"

    # input information
    travelMode = "Driving"
    startTime = "2021-11-15-07:00"
    distanceUnit = "miles"
    timeUnit = "minute"

    routeUrl = (
        "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?"
        + f"origins={orig}"
        + f"&destinations={dest}"
        + f"&travelMode={travelMode}"
        + f"&startTime={startTime}"
        + f"&timeUnit={timeUnit}"
        + f"&key={bingMapsKey}"
    )

    # request = urllib.request.Request(routeUrl)
    # response = urllib.request.urlopen(request, proxies={"http":"http://myproxy:3129"})
    # data = json.load(response)

    response = requests.get(routeUrl)
    data = response.json()

    distance, time = build_od_matrix(data, (size_x, size_y))
    return distance, time

In [30]:
def build_od_matrix(data, size):

    resources = data["resourceSets"][0]["resources"][0]["results"]

    distance = np.zeros((size[0], size[1]))
    time = np.zeros((size[0], size[1]))

    for i in range(size[0]):
        for j in range(size[1]):
            distance[i, j] = resources[i * size[1] + j]["travelDistance"]
            time[i, j] = resources[i * size[1] + j]["travelDuration"]

    return distance, time
    

# Loading the data

In [31]:
# load the housing data
residences = pd.read_csv('../HousingData.csv')
mit_buildings = pd.read_csv('../MIT_Buildings.csv')
shuttle_stops = pd.read_csv("../current_stops.csv").rename(columns={"Station Name": "Name"})

origins = residences[["Residence", "Long", "Lat"]].rename(columns={"Residence": "Name"})
destinations = shuttle_stops.append(residences[["Residence","Lat","Long"]].rename(columns={"Residence":"Name"}))
destinations = destinations.append(mit_buildings[["Name","Lat","Long"]])

# save origins and destionations to CSV
origins.to_csv("../origins.csv", index=False)
destinations.to_csv("../destinations.csv", index=False)

/var/folders/hk/nxk7bnsj4kq9nwsmnmdtzhb80000gn/T/ipykernel_18201/453920896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destinations = shuttle_stops.append(residences[["Residence","Lat","Long"]].rename(columns={"Residence":"Name"}))
/var/folders/hk/nxk7bnsj4kq9nwsmnmdtzhb80000gn/T/ipykernel_18201/453920896.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destinations = destinations.append(mit_buildings[["Name","Lat","Long"]])


In [32]:
N_o = origins.shape[0]
N_d = destinations.shape[0]

# Matrix Initialization
distance_matrix = np.zeros((N_o, N_d))
time_matrix = np.zeros((N_o, N_d))

In [33]:
# Compute the list of origin-destinations points
long = list(origins["Long"])
lat = list(origins["Lat"])

or_points = []
for i, j in zip(lat, long):
    or_points.append(str(i) + "," + str(j))

# Compute the list of origin-destinations points
long = list(destinations["Long"])
lat = list(destinations["Lat"])

des_points = []
for i, j in zip(lat, long):
    des_points.append(str(i) + "," + str(j))

or_points = ";".join(or_points)
des_points = ";".join(des_points)

In [34]:
distance_matrix, time_matrix = get_data(
    or_points, des_points, size_x=int(N_o), size_y=int(N_d)
)

In [35]:
distances = pd.DataFrame(distance_matrix)

# give a destination to each column name
distances.columns = destinations["Name"]
distances.index = origins["Name"]

# Reshape dataframe to long dataframe
distances = distances.reset_index()
distances = pd.melt(distances, id_vars=['Name'], value_vars=distances.columns.values.tolist()[1:], var_name='Destination', value_name='Distance')

In [36]:
times = pd.DataFrame(time_matrix)

# give a destination to each column name
times.columns = destinations["Name"]
times.index = origins["Name"]

# Reshape dataframe to long dataframe
times = times.reset_index()
times.to_csv("../origin_destination_times.csv", index=False)
times = pd.melt(times, id_vars=['Name'], value_vars=times.columns.values.tolist()[1:], var_name='Destination', value_name='Time (min)')

In [37]:
distances["Time (min)"] = times["Time (min)"]
distances = distances.rename(columns={"Name": "Origin"})
distances.head()

,Origin,Destination,Distance,Time (min)
0,Baker House,Kendall Square T,2.752,7.6167
1,Burton Conner,Kendall Square T,2.578,8.0167
2,East Campus,Kendall Square T,0.520,1.8500
3,MacGregor House,Kendall Square T,2.497,7.9667
4,Maseeh Hall,Kendall Square T,1.124,3.9333


In [38]:
distances.to_csv("../origin_destination_distances.csv", index=False)

In [39]:
times

,Name,Destination,Time (min)
0,Baker House,Kendall Square T,7.6167
1,Burton Conner,Kendall Square T,8.0167
2,East Campus,Kendall Square T,1.8500
3,MacGregor House,Kendall Square T,7.9667
4,Maseeh Hall,Kendall Square T,3.9333
...,...,...,...
915,Sidney-Pacific,MIT Plasma Science and Fusion Center,1.7333
916,Tang Hall,MIT Plasma Science and Fusion Center,4.8167
917,The Warehouse,MIT Plasma Science and Fusion Center,1.3000
918,Westgate Apartments,MIT Plasma Science and Fusion Center,4.1167


In [40]:
times = pd.DataFrame(time_matrix)
times.columns = destinations["Name"]
times.index = origins["Name"]

# Reshape dataframe to long dataframe
times = times.reset_index()